In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import logging
from horgues3.dataset import HorguesDataset
from horgues3.models import HorguesModel, PlackettLuceLoss, WeightedPlackettLuceLoss, ListwiseLoss, PairwiseRankingLoss, CombinedRankingLoss, RankNetLoss
from horgues3.betting import calculate_betting_probabilities, format_betting_results, extract_winning_tickets
from horgues3.metrics import BettingAccuracyMetric, ExpectedValueBettingMetric, BettingCalibrationMetric, RankCorrelationMetric
from horgues3.odds import fetch_odds_data, restructure_odds_data
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime

# ログ設定
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

In [ ]:
total_epochs = 100
batch_size = 128

train_start_ymd = '20140101'
train_end_ymd = '20231231'

val_start_ymd = '20240101'
val_end_ymd = '20241231'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"Using device: {device}")

# 学習データの準備
logger.info('Preparing train dataset...')
train_dataset = HorguesDataset(train_start_ymd, train_end_ymd, max_horses=18)
train_dataset.fetch().prepare().build().fit().transform()

# 検証データの準備
logger.info('Preparing val dataset')
val_dataset = HorguesDataset(val_start_ymd, val_end_ymd, max_horses=18)
val_dataset.fetch().prepare().build().set_params(train_dataset.get_params()).transform()

logger.info("Dataset preparation done.")
logger.info(f"Training samples: {len(train_dataset)}")
logger.info(f"Validation samples: {len(val_dataset)}")

# データローダー
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=0  # Windowsでは0に設定
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=0
)

In [ ]:
rankings = val_dataset.transformed_data['rankings']
mask = val_dataset.transformed_data['mask']

# 的中馬券
winning_tickets = extract_winning_tickets(rankings, mask)

# オッズ
logging.info('Preparing odds data...')
odds_data = fetch_odds_data(val_start_ymd, val_end_ymd)
restructured_odds_data = restructure_odds_data(odds_data, val_dataset.transformed_data['race_id'])

# メトリクス
logging.info('Preparing metrics...')
metric_calculators = [
    BettingAccuracyMetric(rankings, mask, winning_tickets),
    ExpectedValueBettingMetric(rankings, mask, winning_tickets, restructured_odds_data),
    BettingCalibrationMetric(rankings, mask, winning_tickets),
    RankCorrelationMetric(rankings, mask),
]

In [ ]:
logging.info('Preparing model...')

# モデル設定
model = HorguesModel(train_dataset.get_params()).to(device)

# 損失関数とオプティマイザ
criterion = WeightedPlackettLuceLoss()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4,
    betas=(0.9, 0.999),
    eps=1e-8
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=total_epochs,  # エポック数に合わせて調整
    eta_min=1e-6
)



In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """1エポックの学習"""
    model.train()
    total_loss = 0
    num_batches = 0
    grad_norms = []
    
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        
        # データをデバイスに移動
        x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
        x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
        rankings = batch['rankings'].to(device)
        mask = batch['mask'].to(device)
        
        # 予測
        scores = model(x_num=x_num, x_cat=x_cat, mask=mask)
        
        # 損失計算
        loss = criterion(scores, rankings, mask)
        
        # 逆伝播
        loss.backward()
        
        # 勾配ノルムを記録
        total_grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        grad_norms.append(total_grad_norm.item())
        
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        
    return {
        'loss': total_loss / num_batches if num_batches > 0 else 0,
        'grad_norm': {
            'mean': np.mean(grad_norms),
            'max': np.max(grad_norms),
            'min': np.min(grad_norms),
        },
    }

In [ ]:
def validate_epoch(model, dataloader, criterion, metric_calculators, device):
    """1エポックの検証"""
    model.eval()
    total_loss = 0
    num_batches = 0
    
    all_scores = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            # データをデバイスに移動
            x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
            x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
            rankings = batch['rankings'].to(device)
            mask = batch['mask'].to(device)
            
            # 予測
            scores = model(x_num=x_num, x_cat=x_cat, mask=mask)
            
            # 損失計算
            loss = criterion(scores, rankings, mask)

            total_loss += loss.item()
            num_batches += 1

            # メトリクス計算用にスコアを保存
            all_scores.append(scores.cpu().numpy())

    all_scores = np.concatenate(all_scores, axis=0)
    betting_probabilities = calculate_betting_probabilities(all_scores)

    metrics = {}
    for calculator in metric_calculators:
        metrics.update(calculator(betting_probabilities))

    return {
        'loss': total_loss / num_batches if num_batches > 0 else 0,
        **metrics
    }

In [ ]:
def format_metrics(metrics_dict, indent=0):
    """
    メトリクス辞書を整形して表示用の文字列に変換
    
    Args:
        metrics_dict: メトリクス辞書
        indent: インデント数
    
    Returns:
        str: 整形された文字列
    """
    lines = []
    indent_str = "  " * indent
    
    for key, value in metrics_dict.items():
        if isinstance(value, dict):
            lines.append(f"{indent_str}{key}:")
            lines.append(format_metrics(value, indent + 1))
        elif isinstance(value, (int, float)):
            if isinstance(value, float):
                lines.append(f"{indent_str}{key}: {value:.4f}")
            else:
                lines.append(f"{indent_str}{key}: {value}")
        elif hasattr(value, 'item'):  # numpy scalar
            lines.append(f"{indent_str}{key}: {value.item():.4f}")
        else:
            lines.append(f"{indent_str}{key}: {value}")
    
    return "\n".join(lines)

In [ ]:
# 学習ループ
logging.info("Initiating training...")
best_val_loss = float('inf')

history = []

for epoch in range(total_epochs):
    logger.info(f"Epoch {epoch+1}/{total_epochs}")
    epoch_info = {}
    
    # 学習
    epoch_info['train'] = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 検証
    epoch_info['val'] = validate_epoch(model, val_loader, criterion, metric_calculators, device)
    
    # スケジューラーを更新
    scheduler.step()
    
    # 整形されたメトリクスをログ出力
    formatted_metrics = format_metrics(epoch_info)
    logger.info(f"=== Epoch {epoch+1}/{total_epochs} Results ===\n{formatted_metrics}")
    
    # ヒストリに記録
    history.append(epoch_info)

    # ベストモデルの保存
    val_loss = epoch_info['val']['loss']
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'val_loss': val_loss,
            'dataset_params': train_dataset.get_params()
        }, 'best_model.pth')
        logger.info(f"New best model saved with val_loss: {val_loss:.4f}")

logger.info("Training completed!")